# Run 2 Load

In [2]:
%run "./initialize"

In [0]:
spark.sql(f"""INSERT INTO TABLE {staging_schema}.customer (
  CUSTOMER_ID,
  FIRST_NAME,
  LAST_NAME,
  EMAIL,
  DELETE_FLAG,
  LOAD_TIMESTAMP)
VALUES
  (1, 'John', 'Doe', 'jdoe@example.com', NULL, '2023-01-02 10:00:00')
  , (3, 'Alice', 'Green', 'alice.green@example.com', NULL, '2023-01-02 10:00:00')
  , (4, 'Joe', 'Bloggs', 'joe.bloggs@example.com', NULL, '2023-01-02 10:00:00')
  , (10, 'Richard', 'Johnson', 'richard.johnson@example.com', True, '2023-01-02 10:00:00');""")

spark.sql(f"""INSERT OVERWRITE TABLE {staging_schema}.customer_snapshot_source (
  CUSTOMER_ID,
  FIRST_NAME,
  LAST_NAME,
  EMAIL,
  DELETE_FLAG,
  LOAD_TIMESTAMP)
VALUES
  (1, 'John', 'Doe', 'jdoe@example.com', NULL, '2023-01-02 10:00:00')
  , (2, 'Jane', 'Smith', 'jane.smith@example.com', NULL, '2023-01-01 10:00:00')
  , (3, 'Alice', 'Green', 'alice.green@example.com', NULL, '2023-01-02 10:00:00')
  , (4, 'Joe', 'Bloggs', 'joe.bloggs@example.com', NULL, '2023-01-02 10:00:00')
  , (10, 'Richard', 'Johnson', 'richard.johnson@example.com', NULL, '2023-01-01 10:00:00')""")

spark.sql(f"""INSERT INTO TABLE {staging_schema}.customer_address (
  CUSTOMER_ID,
  CITY,
  STATE,
  LOAD_TIMESTAMP)
VALUES
  (2, 'Perth', 'WA', '2023-01-02 10:00:00')
  , (3, 'Sydney', 'NSW', '2023-01-02 10:00:00')""")

In [ ]:
spark.sql(f"""INSERT INTO TABLE {staging_schema}.customer_historical_snapshot_source (
  CUSTOMER_ID,
  FIRST_NAME,
  LAST_NAME,
  EMAIL,
  LOAD_TIMESTAMP)
VALUES
  (6, 'Someone', 'else', 'someone.else@example.com', '2024-04-01 10:00:00')""")

In [ ]:
file_content = """CUSTOMER_ID,FIRST_NAME,LAST_NAME,EMAIL,DELETE_FLAG,LOAD_TIMESTAMP\n
6,Someone,else,someone.else@example.com,,2023-03-01 10:00:00\n
"""

dbutils.fs.put(
  f"{customer_snapshot_file_path}/customer_2024_03_01.csv",
  file_content,
  True)

dbutils.fs.put(
  f"{customer_snapshot_partitioned_file_path}/YEAR=2024/MONTH=03/DAY=01/customer.csv",
  file_content,
  True)

dbutils.fs.put(
  f"{template_samples_customer_file_path}/customer_2024_03_01.csv",
  file_content,
  True)

# Template samples customer_address data for run 2
customer_address_file_content = """CUSTOMER_ID,CITY,STATE,LOAD_TIMESTAMP
2,Perth,WA,2024-03-01 10:00:00
3,Sydney,NSW,2024-03-01 10:00:00
6,Adelaide,SA,2024-03-01 10:00:00
"""

dbutils.fs.put(
  f"{template_samples_customer_address_file_path}/customer_address_2024_03_01.csv",
  customer_address_file_content,
  True
)

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("CUSTOMER_ID", StringType(), True),
    StructField("FIRST_NAME", StringType(), True),
    StructField("LAST_NAME", StringType(), True),
    StructField("EMAIL", StringType(), True),
    StructField("DELETE_FLAG", StringType(), True),
    StructField("LOAD_TIMESTAMP", StringType(), True)
])

data = [
    ["6", "Someone", "else", "someone.else@example.com", "", "2023-03-01 10:00:00"]
]

df = spark.createDataFrame(data, schema=schema)

df.write.parquet(
    f"{customer_snapshot_partitioned_parquet_file_path}/YEAR=2024/MONTH=03/DAY=01/customer.parquet",
    mode="overwrite"
)

In [ ]:
file_content = """CUSTOMER_ID,FIRST_NAME,LAST_NAME,EMAIL,DELETE_FLAG,LOAD_TIMESTAMP\n
1,John,Doe,jdoe@example.com,,2023-01-02 10:00:00\n
3,Alice,Green,alice.green@example.com,,2023-01-02 10:00:00\n
4,Joe,Bloggs,joe.bloggs@example.com,,2023-01-02 10:00:00\n
"""

dbutils.fs.put(
  f"{customer_file_path}/customer_2.csv",
  file_content,
  True)

In [ ]:
spark.sql(f"""INSERT INTO TABLE {staging_schema}.customer_snapshots (
  CUSTOMER_ID,
  FIRST_NAME,
  LAST_NAME,
  EMAIL,
  UPDATE_TIMESTAMP,
  SNAPSHOT_TIMESTAMP,
  SNAPSHOT_VERSION)
VALUES
  (1, 'John', 'Doe', 'jdoe@example.com', '2023-01-02 00:00:00','2023-01-03T00:00:00', 2)
  , (3, 'Alice', 'Green', 'alice.green@example.com', '2023-01-02 00:00:00', '2023-01-03T00:00:00', 2)
  , (4, 'Joe', 'Bloggs', 'j.bloggs@example.com', '2023-01-03 00:00:00', '2023-01-03T00:00:00', 2) 
  """)